In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import shutil
import glob
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.tag as tag
import thor.visualize as visualize

notebook_name = "cpol_demo.ipynb"
__file__ = "/home/ewan/Documents/THOR/examples/cpol_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# Parent directory for saving outputs
base_local = Path("/home/ewan/THOR_output")
start = "2005-11-13T14:00:00"
end = "2005-11-13T16:00:00"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)
era5_pl_options = data.era5.data_options(start=start, end=end)
era5_sl_options = data.era5.data_options(
    start=start, end=end, data_format="single-levels"
)
data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, filename="cpol_era5")

# Create the grid_options dictionary using the first file in the cpol dataset
filepath = data_options["cpol"]["filepaths"][0]
with xr.open_dataset(filepath) as cpol:
    if converted_options["load"]:
        latitude, longitude = [
            cpol.latitude.values.tolist(),
            cpol.longitude.values.tolist(),
        ]
    else:
        lats = cpol["point_latitude"].isel(z=0).values
        lons = cpol["point_longitude"].isel(z=0).values
        latitude, longitude = grid.new_geographic_grid(lats, lons, 0.025, 0.025)
grid_options = grid.create_options(
    name="geographic", latitude=latitude, longitude=longitude
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="cpol_geographic")

# Create the tag_options dictionary
era5_pl_tag_options = data.era5.tag_options()
era5_sl_tag_options = data.era5.tag_options(dataset="era5_sl")
tag_options = option.consolidate_options([era5_pl_tag_options, era5_sl_tag_options])
tag.save_tag_options(tag_options, filename="era5")

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol", tags=["era5_pl", "era5_sl"])
option.save_track_options(track_options, filename="cpol_mcs")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(
        obj,
        save=True,
        style="presentation"
    )
    for obj in ["cell", "anvil", "mcs"]}
visualize_options["middle_cloud"] = visualize.option.runtime_options(
    "middle_cloud", save=True, style="presentation", figure_types=["mask"]
)
visualize.option.save_display_options(visualize_options, filename="runtime_mcs")

2024-07-11 22:42:52,487 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/data_options/cpol_era5.yaml


2024-07-11 22:42:52,577 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid_options/cpol_geographic.yaml
2024-07-11 22:42:52,608 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/tag_options/era5.yaml
2024-07-11 22:42:52,611 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/track_options/cpol_mcs.yaml
2024-07-11 22:42:52,620 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/visualize_options/runtime_mcs.yaml


In [5]:
try:
    shutil.rmtree(base_local / "runs/cpol_demo")
except OSError as e:
    print(f"Error: {e.strerror}")
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    tag_options,
    visualize_options,
    output_directory=base_local / "runs/cpol_demo",
)

2024-07-11 22:51:07,014 - thor.track - INFO - Processing 2005-11-13T14:00:09.
2024-07-11 22:51:07,015 - thor.data.aura - DEBUG - Updating cpol dataset for 2005-11-13T14:00:09.
2024-07-11 22:51:07,016 - thor.data.aura - DEBUG - Converting cpol data from twp10cpolgrid150.b2.20051113.140000.nc
2024-07-11 22:51:07,513 - thor.track - DEBUG - Processing hierarchy level 0.
2024-07-11 22:51:07,514 - thor.track - DEBUG - Tracking cell.
2024-07-11 22:51:07,524 - thor.match.match - DEBUG - Matching cell objects.
2024-07-11 22:51:07,525 - thor.match.match - DEBUG - No previous mask, or no objects in previous mask.
2024-07-11 22:51:07,632 - thor.visualize.runtime - DEBUG - Saving mask figure for cell.
2024-07-11 22:51:08,775 - thor.visualize.runtime - DEBUG - Saving match figure for cell.
2024-07-11 22:51:11,657 - thor.track - DEBUG - Tracking middle_cloud.
2024-07-11 22:51:11,757 - thor.visualize.runtime - DEBUG - Saving mask figure for middle_cloud.
2024-07-11 22:51:12,790 - thor.track - DEBUG - 

         143394533 function calls (138226662 primitive calls) in 254.037 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1066   41.267    0.039   41.351    0.039 {method 'draw_path_collection' of 'matplotlib.backends._backend_agg.RendererAgg' objects}
       14   38.953    2.782   38.953    2.782 {built-in method posix.waitpid}
  5596695   35.251    0.000   35.251    0.000 predicates.py:747(intersects)
  5686200   10.765    0.000   53.475    0.000 decorators.py:62(wrapped)
   492953   10.371    0.000   10.371    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      296    5.725    0.019    5.725    0.019 {method 'encode' of 'ImagingEncoder' objects}
3401659/6858    4.247    0.000   11.625    0.002 copy.py:128(deepcopy)
       13    3.554    0.273    3.554    0.273 cbindings.py:2051(ESMP_FieldRegridStore)
  5596695    3.501    0.000   60.319    0.000 base.py:717(intersects)
    10881    3.228    0.000   16.971    0.002